Script must be run in same directory as data source (i.e., Excel spreadsheet of map labels).

In [1]:
# Define variables; then, run script
filename_of_input_labels_spreadsheet = 'labels_spreadsheet_202304.xlsx' # Include file extension
name_of_field_for_call_number = 'call_number'
name_of_field_for_description = 'description'
name_of_field_for_sheet_range = 'sheet_range'
name_of_field_for_scale = 'scale'
desired_name_for_combined_sheet_range_and_scale_field = 'sheet_range_and_scale'
filename_of_output_labels_spreadsheet = 'labels_manipulated.xlsx' # Include file extension

In [2]:
# Import required libraries
import pandas as pd
import xlsxwriter

In [3]:
# Read labels Excel spreadsheet into a pandas DataFrame
df = pd.read_excel(filename_of_input_labels_spreadsheet)

In [4]:
# Remove square brackets from scale field
df['scale'] = df['scale'].str.replace('[\[\]]', '', regex = True)

In [5]:
# Create a boolean mask where True values indicate rows where both sheet_range and scale fields are not null
range_and_scale_present = df[[name_of_field_for_sheet_range, name_of_field_for_scale]].isna().apply(lambda x: x.sum(), axis = 1) == 0

In [6]:
# Prepend 'Sheets:' and 'Scale:' to values of those fields where both are not null
# Insert a new line after these prepended strings
df.loc[range_and_scale_present, 'sheet_range'] = df.loc[range_and_scale_present, 'sheet_range'].apply(lambda x: 'Sheets:\n' + x)
df.loc[range_and_scale_present, 'scale'] = df.loc[range_and_scale_present, 'scale'].apply(lambda x: 'Scale:\n' + x)

In [7]:
# Combine values of 'sheet_range' and 'scale' fields, using return to separate values on different lines
df[desired_name_for_combined_sheet_range_and_scale_field] = df[[name_of_field_for_sheet_range, name_of_field_for_scale]].apply(
    lambda x: '\n'.join(x.dropna()),
    axis = 1
)

In [8]:
# Drop 'sheet_range' and 'scale' fields, as they are no longer needed
df = df.drop(columns = [name_of_field_for_sheet_range, name_of_field_for_scale])

In [9]:
# Add empty border columns
df.insert(loc = 0, column = 'left_border', value = '')
df.insert(loc = 4, column = 'right_border', value = '')

In [10]:
# Create two separate dataframes (top and bottom) from original
# These will be joined later
top_half = df.iloc[:round(len(df) / 2)]
bottom_half = df.iloc[round(len(df) / 2):].reset_index(drop = True)

In [11]:
# Append '_2' onto column names for bottom half of dataframe
# This will indicate fields that belong to the second column of labels
bottom_half.columns = [column_name + '_2' for column_name in bottom_half.columns]

In [12]:
# Join top and bottom halves of original dataframe
two_columns = top_half.merge(bottom_half, how = 'left', left_index = True, right_index = True)

In [13]:
# Increment each index value by one
two_columns.index = [i + 1 for i in two_columns.index]

In [14]:
# Create empty DataFrame
empty_df = pd.DataFrame(
    index = range(0, len(two_columns)),
    columns = two_columns.columns,
    data = ''
)

In [15]:
# Create second empty DataFrame
empty_df2 = pd.DataFrame(
    index = range(1, len(two_columns) + 1),
    columns = two_columns.columns,
    data = ''
)

In [16]:
# Concatenate two_columns, empty_df, and empty_df2
combined_df = pd.concat([two_columns, empty_df, empty_df2])

In [17]:
# Sort combined_df to intersperse empty rows from empty_df
combined_df = combined_df.sort_index(kind = 'stable')

#### Write resulting DataFrame back to Excel file

In [18]:
# Create a pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(filename_of_output_labels_spreadsheet, engine = 'xlsxwriter')

In [19]:
# Convert the dataframe to an XlsxWriter Excel object
combined_df.to_excel(writer, sheet_name = 'map_drawer_labels', index = False, header = False)

In [20]:
# Define XlsxWriter workbook and worksheet objects
workbook  = writer.book
worksheet = writer.sheets['map_drawer_labels']

In [21]:
# Configure cell formatting
cells_without_borders = workbook.add_format({
    'align': 'center',
    'valign': 'vcenter',
    'font_name': 'Calibri',
    'font_size': 16,
    'font_color': 'black',
    'text_wrap': True,
    'shrink': True
})

right_border = workbook.add_format({'right': 3})

bottom_border = workbook.add_format({'bottom': 3})

In [22]:
# Set row heights for each label
# Width is second argument in .set_row() method and is in character units
worksheet.set_row(0, 5)

for row in range(1, len(combined_df) + 1, 3):
    worksheet.set_row(row, 70, cells_without_borders)

for row in range(2, len(combined_df) + 1, 3):
    worksheet.set_row(row, 10, bottom_border)

for row in range(3, len(combined_df), 3):
    worksheet.set_row(row, 10)

In [23]:
# Set column widths for each label
# Width is third argument and is in character units
worksheet.set_column(0, 0, 0) # 'left_border' column
worksheet.set_column(1, 1, 17) # 'call_number' column
worksheet.set_column(2, 2, 20) # 'description' column
worksheet.set_column(3, 3, 17) # 'sheet_range_and_scale' column
worksheet.set_column(4, 4, 2, right_border) # 'right_border' column
worksheet.set_column(5, 5, 2) # 'left_border_2' column
worksheet.set_column(6, 6, 17) # 'call_number_2' column
worksheet.set_column(7, 7, 20) # 'description_2' column
worksheet.set_column(8, 8, 17) # 'sheet_range_and_scale_2' column
worksheet.set_column(9, 9, 2, right_border) # 'right_border_2' column

0

In [24]:
# Set paper size to 11 in. x 17 in.
worksheet.set_paper(3)

In [25]:
# Set page breaks
worksheet.set_h_pagebreaks(range(30, len(combined_df), 30))

In [26]:
# Set margins to zero
worksheet.set_margins(
    left = 0,
    right = 0,
    top = 0,
    bottom = 0
)

In [27]:
# Set the cells to be printed
worksheet.print_area(0, 0, len(combined_df) + 1, 9)

0

In [28]:
# Close the pandas Excel writer, and output the Excel file
writer.close()